### Food Ordering Chatbot using NLP, Logistic Regression, and TF-IDF Vectorizer

This notebook demonstrates the creation of a simple food ordering chatbot using machine learning techniques. The chatbot is designed to assist users in placing food orders by understanding natural language inputs and generating responses based on predefined intents and patterns.

### The key components of the chatbot include:

### Intents and Patterns:
Predefined categories (intents) like greetings, placing orders, and providing personal details (name, phone number, and address). Each intent has a set of sample user inputs (patterns) that the model uses for training.

### TF-IDF Vectorization: 
The user inputs are transformed into numerical form using the TfidfVectorizer from scikit-learn, which helps in understanding the text's importance and context.

### Logistic Regression: 
A machine learning classifier that is trained on the intents and patterns. It predicts the user's intent from their input and responds accordingly.

### Regular Expressions (re module): 
Used to extract details like the user's name, address, and quantity of the order.

The chatbot follows a simple conversational flow, guiding the user through the process of placing an order, asking for necessary details, and confirming the order at the end.



In [1]:
import random
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Define intents and responses for food ordering
intents = [
    {"tag": "greeting", "patterns": ["Hi", "Hello", "Hey", "How are you"], "responses": ["Hi there! How can I help you with your food order today?", "Hello! Ready to take your order.", "Hey! What would you like to order today?"]},
    {"tag": "order", "patterns": ["I want to order", "I'd like to order", "Can I place an order", "I would like to order"], "responses": ["Sure! I'll need some details to complete your order."]},
    {"tag": "name", "patterns": ["My name is", "I am", "Call me"], "responses": ["Got it, {name}. What would you like to order?"]},
    {"tag": "phone", "patterns": ["My phone number is", "Call me at", "You can reach me at"], "responses": ["Thank you! I have noted your phone number."]},
    {"tag": "address", "patterns": ["My address is", "I live at", "Deliver to"], "responses": ["Got it, I'll deliver to {address}."]},
    {"tag": "order_details", "patterns": ["I want", "I would like", "I'll have", "Get me"], "responses": ["Great choice! How many of these would you like?"]},
    {"tag": "quantity", "patterns": ["Quantity", "Number", "How many"], "responses": ["Thank you! Your order is confirmed."]},
    {"tag": "thanks", "patterns": ["Thank you", "Thanks", "I appreciate it"], "responses": ["You're welcome! Your order will be delivered shortly."]},
    {"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye", "Take care"], "responses": ["Goodbye! Have a great day!"]}
]

# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# Train the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

# Define the chatbot function
def chatbot(input_text, user_info):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    
    response = ""
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            if "{name}" in response:
                response = response.format(name=user_info.get('name', ''))
            if "{address}" in response:
                response = response.format(address=user_info.get('address', ''))
            return response
    return "Sorry, I didn't understand that. Could you please provide more details?"

# Define the chat function
def chat():
    user_info = {}
    state = "greeting"
    print("Chatbot: Hi! How can I assist you with your food order today?")
    
    while True:
        user_input = input("You: ").strip().lower()
        
        if user_input in ['exit', 'quit', 'bye']:
            print("Chatbot: Goodbye! Have a great day!")
            break
        
        if state == "greeting" and "order" in user_input:
            print("Chatbot: Sure! I'll need some details to complete your order.")
            state = "name"
        elif state == "name":
            if "name is" in user_input:
                user_info['name'] = user_input.split('name is ')[-1].strip()
                print(f"Chatbot: Got it, {user_info['name']}. What would you like to order?")
                state = "order_details"
            else:
                print("Chatbot: Please provide your name in the format 'My name is [Your Name]'.")
        elif state == "order_details":
            match = re.search(r'(?:i want |i would like |i\'ll have |get me )(.+)', user_input)
            if match:
                user_info['order'] = match.group(1).strip()
                print("Chatbot: Great choice! How many of these would you like?")
                state = "quantity"
            else:
                print("Chatbot: Please specify your order in the format 'I want [Dish Name]'.")
        elif state == "quantity":
            if re.search(r'\b\d+\b', user_input):
                user_info['quantity'] = re.search(r'\b\d+\b', user_input).group()
                print("Chatbot: Thank you! Your order is confirmed.")
                print(f"Chatbot: Order Summary - Name: {user_info.get('name')}, Phone: {user_info.get('phone')}, Address: {user_info.get('address')}, Order: {user_info.get('order')}, Quantity: {user_info.get('quantity')}")
                print("Chatbot: Your order will be delivered shortly.")
                break
            else:
                print("Chatbot: Please provide the quantity in a numeric format.")
        elif state == "phone":
            if "phone number is" in user_input:
                user_info['phone'] = user_input.split('phone number is ')[-1].strip()
                print("Chatbot: Thank you! I have noted your phone number.")
                state = "address"
            else:
                print("Chatbot: Please provide your phone number in the format 'My phone number is [Your Phone Number]'.")
        elif state == "address":
            if "address is" in user_input:
                user_info['address'] = user_input.split('address is ')[-1].strip()
                print(f"Chatbot: Got it, I'll deliver to {user_info['address']}.")
                state = "order_details"
            else:
                print("Chatbot: Please provide your address in the format 'My address is [Your Address]'.")
        else:
            response = chatbot(user_input, user_info)
            print(f"Chatbot: {response}")

# Run the chatbot
chat()


Chatbot: Hi! How can I assist you with your food order today?


You:  hi


Chatbot: Hi there! How can I help you with your food order today?


You:  order food


Chatbot: Sure! I'll need some details to complete your order.


...

Chatbot: Got it, kapish. What would you like to order?


You:  i want t order pizza


Chatbot: Great choice! How many of these would you like?


You:  2


Chatbot: Thank you! Your order is confirmed.
Chatbot: Order Summary - Name: kapish, Phone: None, Address: None, Order: t order pizza, Quantity: 2
Chatbot: Your order will be delivered shortly.
